# Objective: Toeknization of SMILES data to predict Binding Affinity with CHEMBert Model

Some papers: 
SMILES-BERT: Large Scale Unsupervised Pre-Training for Molecular Property Prediction
https://dl.acm.org/doi/abs/10.1145/3307339.3342186?casa_token=x0RGiJZbcM8AAAAA:Jpdz0mKm8oQAjhXj4Z0BDEbcR4WoZ8v3113op2iUXx1I4erUs4t6i7ubPdBPNe5A4pN0LUiPuFveog

PySMILESUtils – Enabling deep learning with the SMILES chemical language:
chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://chemrxiv.org/engage/api-gateway/chemrxiv/assets/orp/resource/item/60d99ad2fca490cddfc97083/original/py-smiles-utils-enabling-deep-learning-with-the-smiles-chemical-language.pdf


![](https://ars.els-cdn.com/content/image/1-s2.0-S2001037021000763-gr4.jpg)

https://www.sciencedirect.com/science/article/pii/S2001037021000763

## 1. Import libraries

In [1]:
!pip install rdkit
!pip install duckdb
!pip install transformers pandas torch

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder

The training set is pretty big, but we can treat the parquet files as databases using duckdb. We will use this to sample down to a smaller dataset for demonstration purposes. Lets install duckdb as well.

## 2. Data Preparation (Train & Test)

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives. 

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

--> changed to 100

In [3]:
import duckdb
import pandas as pd

train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
df.head(10)

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,242610268,O=C(O)COc1cccc(-c2csc(NC(=O)OCC3c4ccccc4-c4ccc...,NCC1CCC(C(F)F)CC1,Cl.NC1CCC(=O)CC1,O=C1CCC(Nc2nc(NCC3CCC(C(F)F)CC3)nc(Nc3nc(-c4cc...,HSA,0
1,239782150,O=C(O)CNC(=O)OCC1c2ccccc2-c2ccccc21,CCN1CCN(Cc2ccc(N)nc2)CC1,Nc1cc(F)cc(F)c1[N+](=O)[O-],CCN1CCN(Cc2ccc(Nc3nc(NCC(=O)N[Dy])nc(Nc4cc(F)c...,HSA,0
2,11752955,CC(=O)c1ccc(C[C@H](NC(=O)OCC2c3ccccc3-c3ccccc3...,Nc1cc(Cl)cnc1Cl,Cc1nc(N)ccc1[N+](=O)[O-],CC(=O)c1ccc(C[C@H](Nc2nc(Nc3ccc([N+](=O)[O-])c...,sEH,0
3,46163777,Cc1c(Br)ccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,Cc1ccccc1-c1csc(N)n1,Cc1cc(CN)c(C)o1,Cc1cc(CNc2nc(Nc3nc(-c4ccccc4C)cs3)nc(Nc3c(C(=O...,sEH,0
4,50566520,Cc1cc(C(=O)O)ccc1NC(=O)OCC1c2ccccc2-c2ccccc21,Cc1sc(C)c(CN)c1Br.Cl,Nc1nc(Br)cn2ccnc12,Cc1cc(C(=O)N[Dy])ccc1Nc1nc(NCc2c(C)sc(C)c2Br)n...,sEH,0
5,40479064,COc1nccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,CNC(=O)c1ccc(N)cc1F,Nc1cncc(F)c1,CNC(=O)c1ccc(Nc2nc(Nc3cncc(F)c3)nc(Nc3c(C(=O)N...,HSA,0
6,214520933,O=C(Nc1cccc(I)c1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1cccc(CN2CCCCC2)c1,Nc1nncs1,O=C(N[Dy])c1c(I)cccc1Nc1nc(Nc2cccc(CN3CCCCC3)c...,sEH,0
7,261042013,O=C(O)C[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)c1c...,Cl.NCc1csc(=O)[nH]1,NCCc1ccc(N2CCOCC2)c(F)c1,O=C(C[C@@H](Nc1nc(NCCc2ccc(N3CCOCC3)c(F)c2)nc(...,HSA,0
8,32915901,COc1cc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)c...,COc1c(F)ccc(F)c1CN.Cl,Cc1cnc(N)s1,COc1cc(Nc2nc(NCc3c(F)ccc(F)c3OC)nc(Nc3ncc(C)s3...,BRD4,0
9,38421201,COc1cccc(C(=O)O)c1NC(=O)OCC1c2ccccc2-c2ccccc21,COc1cnc(N)cn1,Cl.NCCn1cnc2sccc2c1=O,COc1cnc(Nc2nc(NCCn3cnc4sccc4c3=O)nc(Nc3c(OC)cc...,BRD4,0


In [7]:
df.shape

(60000, 7)

## 3. Tokenization using SmilesPE & creating DataLoader


In [8]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q SmilesPE
!pip install --upgrade deepsmiles

Example usage of SmilesPE

In [11]:
from SmilesPE.pretokenizer import atomwise_tokenizer

smi = 'CC[N+](C)(C)Cc1ccccc1Br'
toks = atomwise_tokenizer(smi)
print(toks)

['C', 'C', '[N+]', '(', 'C', ')', '(', 'C', ')', 'C', 'c', '1', 'c', 'c', 'c', 'c', 'c', '1', 'Br']


Functions

In [ ]:
def tokenize_smiles(smiles, tokenizer):
    if pd.isna(smiles):
        return []
    return tokenizer.tokenize(smiles)


In [38]:
# Padding and attention mask generation
def pad_sequences(sequences, max_length=None, padding_value=0):
    if not max_length:
        max_length = max(len(seq) for seq in sequences)
    
    padded_sequences = []
    attention_masks = []
    
    for seq in sequences:
        # Padding the sequence
        padded_seq = seq + [padding_value] * (max_length - len(seq))
        padded_sequences.append(padded_seq)
        
        # Attention mask: 1 for real tokens, 0 for padding
        attention_mask = [1] * len(seq) + [0] * (max_length - len(seq))
        attention_masks.append(attention_mask)
    
    return padded_sequences, attention_masks


In [39]:
# Apply padding and generate attention masks for the train and validation sets
train_texts_padded, train_attention_mask = pad_sequences(train_texts)
val_texts_padded, val_attention_mask = pad_sequences(val_texts)

# Convert padded sequences and attention masks into tensors
train_texts_tensor = torch.tensor(train_texts_padded)
val_texts_tensor = torch.tensor(val_texts_padded)
train_attention_mask_tensor = torch.tensor(train_attention_mask)
val_attention_mask_tensor = torch.tensor(val_attention_mask)

# Convert train_labels and val_labels to tensors (as before)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)  # For classification
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)      # For classification


**Dataloader generation**

In [41]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Function to convert tokenized SMILES into sequences of indices
def tokens_to_indices(tokens, vocab):
    return [vocab[token] for token in tokens if token in vocab]

# Convert tokenized SMILES into sequences of indices
df['indexed_smiles'] = df['tokenized_smiles'].apply(lambda x: tokens_to_indices(x, vocab))

# Split data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['indexed_smiles'], df['binds'], test_size=0.2)

# Padding and truncation: Since the tokens are sequences of varying length, we'll pad them to the same length
def pad_sequences(sequences, max_length=None, padding_value=0):
    if not max_length:
        max_length = max(len(seq) for seq in sequences)
    padded_sequences = [seq + [padding_value] * (max_length - len(seq)) for seq in sequences]
    return padded_sequences

# Apply padding to the train and validation texts
train_texts_padded = pad_sequences(train_texts)
val_texts_padded = pad_sequences(val_texts)

# Convert the padded sequences into tensors
train_texts_tensor = torch.tensor(train_texts_padded)
val_texts_tensor = torch.tensor(val_texts_padded)

# Convert train_labels and val_labels from pandas Series to lists (if they are pandas Series)
train_labels = train_labels.tolist()
val_labels = val_labels.tolist()

# Convert labels to tensors
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)  # Use torch.long for classification labels
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)      # Use torch.long for classification labels


class SmilesDataset(Dataset):
    def __init__(self, encodings, attention_masks, labels):
        self.encodings = encodings
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings[idx].clone().detach(),          # Clone and detach the tensor
            'attention_mask': self.attention_masks[idx].clone().detach(),  # Clone and detach the tensor
            'labels': self.labels[idx].clone().detach()                 # Clone and detach the tensor
        }

    def __len__(self):
        return len(self.labels)



In [42]:
# Create datasets
train_dataset = SmilesDataset(train_texts_tensor, train_attention_mask_tensor, train_labels_tensor)
val_dataset = SmilesDataset(val_texts_tensor, val_attention_mask_tensor, val_labels_tensor)

In [43]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [44]:
# Inspect the first item in the val_dataset
first_item = val_dataset[0]
print("First Item Input IDs (Tokenized SMILES):", first_item['input_ids'])
print("First Item Label (Binds):", first_item['labels'])


First Item Input IDs (Tokenized SMILES): tensor([ 6, 26,  3,  7,  4,  7,  7,  5,  6,  7,  8,  9,  7,  5,  6,  7, 10,  7,
         7,  5,  6, 14,  3,  3,  6,  3,  3, 14, 12,  7,  7,  7, 10, 17,  5,  2,
         1, 12, 18, 12,  9,  7,  5,  6, 24,  5,  3,  3,  5,  2,  1, 12,  6, 15,
        12,  3,  7, 10,  7,  7,  7,  5, 17,  5,  2,  1, 12, 18, 12,  7,  7, 10,
        12,  9,  8, 12,  7,  7,  7,  4, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
First Item Label (Binds): tensor(1)


# 4. Model Training with CHEMBert Model

In [36]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the ChemBERTa tokenizer and model for classification
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
model = AutoModelForSequenceClassification.from_pretrained("DeepChem/ChemBERTa-77M-MLM", num_labels=2)  # Assuming binary classification

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
print(f"Length of train_texts_padded: {len(train_texts_padded)}")
print(f"Length of train_labels: {len(train_labels)}")

Length of train_texts_padded: 48000
Length of train_labels: 48000


In [46]:
print(type(train_texts_padded))
print(type(train_labels))

<class 'list'>
<class 'list'>


In [ ]:
from transformers import AdamW
from torch import nn

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Training loop with accuracy calculation
for epoch in range(3):  # Train for 3 epochs
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)  # Include attention mask
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # Extract loss and logits from the model outputs
        loss = outputs.loss
        logits = outputs.logits  # Get the logits (raw predictions)
    
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        _, predicted_labels = torch.max(logits, dim=1)  # Get the index of the max logit (predicted class)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)  # Count the number of samples in the batch

    # Calculate average loss and accuracy for the epoch
    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_samples

    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Training Accuracy = {accuracy:.4f}")


In [ ]:
df['buildingblock1_tokens'] = df['buildingblock1_smiles'].apply(lambda x: tokenize_smiles(x, spe))
df['buildingblock2_tokens'] = df['buildingblock2_smiles'].apply(lambda x: tokenize_smiles(x, spe))
df['buildingblock3_tokens'] = df['buildingblock3_smiles'].apply(lambda x: tokenize_smiles(x, spe))
df['molecule_tokens'] = df['molecule_smiles'].apply(lambda x: tokenize_smiles(x, spe))